# Sexto notebook

En este penultimo notebook se  analiza el desempeño real del modelo entrenado para decidir
si el bajo rendimiento (si existe) se debe a:
- umbrales mal configurados
- clases confundidas
- pocos datos
- o necesidad de más entrenamiento

Este notebook:
- Evalúa el modelo con distintos thresholds
- Analiza predicciones incorrectas
- Resume errores por clase
- Ayuda a decidir si cambiar a dataset completo


In [ ]:
# --- IMPORTS ---
from pathlib import Path
import json
from collections import defaultdict, Counter


In [ ]:
# --- RUTAS DEL PROYECTO ---
#rutas creadas en los previos notebooks
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent

MODELS_DIR = PROJECT_ROOT / "models"
YOLO_MODEL_PATH = MODELS_DIR / "yolo_best.pt"

YOLO_DATASET_DIR = PROJECT_ROOT / "data" / "processed" / "yolo_dataset"
DATA_YAML_PATH = YOLO_DATASET_DIR / "data.yaml"

ARTIFACTS_DIR = PROJECT_ROOT / "artifacts"
CONFIG_SNAPSHOT_PATH = ARTIFACTS_DIR / "config_snapshot.json"

print("Modelo:", YOLO_MODEL_PATH)
print("Dataset:", DATA_YAML_PATH)


In [ ]:
# --- VALIDACIONES ---
#es fundamental que ya exista el modelo entrenado y el yaml por eso de debe validar que existen y son consistentes
def assert_exists(p: Path, desc: str) -> None:
    if not p.exists():
        raise FileNotFoundError(f"Falta {desc}: {p}")

assert_exists(YOLO_MODEL_PATH, "modelo entrenado")
assert_exists(DATA_YAML_PATH, "data.yaml")

print("Validaciones OK.")


In [ ]:
# --- CARGA DE CONFIGURACIÓN ---
#carga del archivo .json del modelo yolo
with open(CONFIG_SNAPSHOT_PATH, "r", encoding="utf-8") as f:
    cfg = json.load(f)

cfg


1. Validacion del modelo 

In [ ]:
# --- IMPORTAR YOLO ---
from ultralytics import YOLO

model = YOLO(str(YOLO_MODEL_PATH))


In [ ]:
# --- EVALUACIÓN BASE (THRESHOLDS POR DEFECTO) ---
#Validacion de que si se haya cumpleido los parametros inciiales del rpimer notebook con respetco al consumo de datos, tamanano de imagenes y batch a usarse
base_metrics = model.val(
    data=str(DATA_YAML_PATH),
    imgsz=cfg["img_size"],
    conf=cfg["conf_threshold"],
    iou=cfg["iou_threshold"],
    batch=cfg["batch_size"]
)

print("Evaluación base completada.")


2. Evaluación con distintos thresholds

Aquí probamos diferentes valores de `conf` para ver
cómo cambian precision y recall sin reentrenar el modelo.

In [ ]:
# --- BARRIDO DE CONFIDENCE THRESHOLD ---
conf_values = [0.1, 0.25, 0.4, 0.6]
conf_results = {}

for conf in conf_values:
    results = model.val(
        data=str(DATA_YAML_PATH),
        imgsz=cfg["img_size"],
        conf=conf,
        iou=cfg["iou_threshold"],
        batch=cfg["batch_size"],
        verbose=False
    )
    conf_results[conf] = {
        "precision": float(results.results_dict.get("metrics/precision(B)", 0)),
        "recall": float(results.results_dict.get("metrics/recall(B)", 0)),
        "mAP50": float(results.results_dict.get("metrics/mAP50(B)", 0))
    }

conf_results


## Interpretación rápida

- Si al bajar `conf` mejora mucho el recall → faltan detecciones
- Si al subir `conf` mejora precision → hay muchos falsos positivos
- Si todo es bajo → el problema es **datos o entrenamiento**, no thresholds
